# Azure OpenAI - Function Calling to retrieve Real-time weather of a city
The latest versions of gpt-35-turbo and gpt-4 have been fine-tuned to work with functions and are able to both determine when and how a function should be called.This sample contains Jupyter notebook to test functional calling first-hand by retrieving real time weather of a city from https://rapidapi.com/weatherapi/api/weatherapi-com.

At a high level you can break down working with functions into three steps:

1. Call the chat completions API with your functions and the user’s input
2. Use the model’s response to call your API or function
3. Call the chat completions API again, including the response from your function to get a final response



### Pre-requisites:
1. Install the latest version of openai Python package and set openai.api_version code variable to July 2023 version or above

    ``` Python
        openai.api_version = "2023-07-01-preview"
    ```
2. Retrieve Azure Open AI model deployment name, API endpoint and API key from Azure OpenAI settings and assign them to "OPENAI_DEPLOYMENT_NAME", "OPENAI_API_BASE" and "OPENAI_API_KEY" environment variables, respectively;
3. Generator your Rapid API key from https://rapidapi.com/ and assign it to "RAPID_API_KEY" environment variable.

In [16]:
# Import required packages
import openai
import os
import json
import datetime
import requests
from pathlib import Path
from dotenv import load_dotenv

BASE_DIR = Path(".")
ENV_DIR = BASE_DIR / "env"
DOT_ENV = ENV_DIR / ".env"

load_dotenv(DOT_ENV, override=True) # Load environment variables from .env file

# Define Azure OpenAI endpoint parameters
openai.api_type = "azure"
openai.api_version = "2023-07-01-preview"
openai.api_base = os.getenv("OPENAI_API_BASE") # Set AOAI endpoint value as env variable
openai.api_key = os.getenv("OPENAI_API_KEY") # Set AOAI API key as env variable
aoai_deployment_name = os.getenv("OPENAI_DEPLOYMENT_NAME") # Set AOAI deployment name as env variable

# Define Rapid API parameter //https://rapidapi.com/collection/recommended-apis

rapid_api_key = os.getenv("RAPID_API_KEY") # Set Rapid API key as env variable
rapid_api_host = "weatherapi-com.p.rapidapi.com" # Set Rapid API host for WeatherAPI.com as env variable

### 1. Function to get Realtime Weather from WeatherAPI(rapidapi.com):
We'll use get_current_weather custom function to interact with weather API and retrieve the real time weather of a given city. JSON structure returned by API endpoint is parsed to extract only specific values.

In [17]:

# Function to get stock price from Alpha Vantage API
def get_current_weather(city):
    today = datetime.date.today().strftime("%Y-%m-%d")
    print(f"Getting current weather for {city} on {today}")
    try:
        url = "https://weatherapi-com.p.rapidapi.com/current.json"
        querystring = {"q":{city}}
        headers = {
    
	"X-RapidAPI-Key": rapid_api_key,
	"X-RapidAPI-Host": rapid_api_host
}
        response = requests.get(url, headers=headers, params=querystring)
        current_temp_inc = response.json()["current"]["temp_c"]
        current_temp_inf = response.json()["current"]["temp_f"]
        current_wind_kph = response.json()["current"]["wind_kph"]
        current_humidity = response.json()["current"]["humidity"]
        current_condition = response.json()["current"]["condition"]["text"]
        current_wind_direction = response.json()["current"]["wind_dir"]
        return str(current_temp_inc), str(current_temp_inf), str(current_wind_kph), str(current_humidity), str(current_condition), str(current_wind_direction)
    except:
        return "NA", "NA", "NA","NA", "NA", "NA"

### 2. JSON structure for GPT function definition:
Function calling is available in the `2023-07-01-preview` API version and works with version `0613` of `gpt-35-turbo`, `gpt-35-turbo-16k`, `gpt-4`, and `gpt-4-32k`.

A function has three main parameters: `name`, `description`, and `parameters`. The `description` parameter is used by the model to determine when and how to call the function so it's important to give a meaningful description of what the function does.

`parameters` is a JSON schema object that describes the parameters that the function accepts. You can learn more about JSON schema objects in the JSON schema reference.

If you want to describe a function that doesn't accept any parameters, use `{"type": "object", "properties": {}}` as the value for the `parameters` property.

In [18]:

functions = [
    {
        "name": "get_current_weather",
        "description": "Retrieve the current weather of the given city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "name of the city, for example Indore"
                }
            },
            "required": ["city"],
        }   
    }
]

available_functions = {
    "get_current_weather": get_current_weather,
} 

### 3. Helper function to interact with Azure OpenAI model:

To use function calling with the Chat Completions API, you need to include two new properties in your request: `functions` and `function_call`. You can include one or more functions in your request.

When functions are provided, by default the `function_call` will be set to `"auto"` and the model will decide whether or not a function should be called. Alternatively, you can set the `function_call` parameter to `{"name": "<insert-function-name>"}` to force the API to call a specific function or you can set the parameter to "none" to prevent the model from calling any functions.

The response from the API includes a `function_call` property if the model determines that a function should be called. The function_call property includes the name of the function to call and the arguments to pass to the function. The arguments are a JSON string that you can parse and use to call your function.

  ```json
  {
    "arguments": "{\n\"city\": \"indore\"\n}",
    "name": "get_current_weather"
  }
  ```



In [19]:
def run_conversation(messages, functions, available_functions, deployment_id):
    
    response = openai.ChatCompletion.create(
        deployment_id=aoai_deployment_name,
        messages=messages,
        functions=functions,
        function_call="auto", 
    )
    response_message = response["choices"][0]["message"]

    # Check if the model wants to call a function
    if response_message.get("function_call"):
        print("Recommended function call:")
        print(response_message.get("function_call"))
        print()
        
        # Note: the JSON response may not always be valid; be sure to handle errors    
           
        function_name = response_message["function_call"]["name"]
        
        # Verify that function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name] 
        
        # Verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        print(function_args)
        current_temp_inc1,current_temp_inf1,current_wind_kph1,current_humidity1,current_condition1,current_wind_direction1 = function_to_call(**function_args)
        function_response =  f"Temprature in degree celsius:{current_temp_inc1}, Temprature in fahrenheit:{current_temp_inf1}, Current wind speed in kph:{current_wind_kph1}, Current humidity:{current_humidity1}, Weather condition is:{current_condition1}, Wind direction is:{current_wind_direction1}",
        
        print("Output of function call:")
        print(function_response)
        print()
        
        # Adding assistant response to messages
        messages.append(
            {
                "role": "assistant",
                "name": function_name,
                "content": response_message["function_call"]["arguments"],
            }
        )

        # Adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": str(function_response),
            }
        )

        # Call the API again to get the final response from the model
        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=aoai_deployment_name
        )  
        return second_response

### 4. Testing:

The last cell in the notebook helps to test everything end to end. We will request for a real time weather of a specific city by taking a user input.

In [ ]:
user_input = input() 
messages=[
        {"role": "system", 
         "content": """You're an AI assistant designed to help users ask for weather. 
         When a user asks for weather of specific city, you should call the get_current_weather function. 
         If the question is not related to weather of the city, you should respond with a polite message.         
         Only use the functions you have been provided with.
         politely say NO if you dont know the answer to a question.
         """},
        {"role": "user", "content": user_input}
    ]
assistant_response = run_conversation(messages, functions, available_functions, aoai_deployment_name)
print(assistant_response['choices'][0]['message'])